### Imports

In [4]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [5]:
%load_ext autoreload
%autoreload 2

In [12]:
import sys
from pathlib import Path
scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))

scripts_path = Path("../Evaluation/").resolve()
sys.path.append(str(scripts_path))

In [16]:
import pickle
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from scripts.data_visualiser import *
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.get_statistics import *
from model_scripts.pre_trained_temporal import *
from model_scripts.dataset_creation import *
from model_scripts.train_model_ae import *
from model_scripts.model_visualiser import *
from model_scripts.clustering import *
from model_scripts.subpatch_extraction import *
from evaluation_scripts.result_visualiser import *
from evaluation_scripts.label_helper import *
from Pipeline.pre_processing_pipeline import *
from Pipeline.temporal_preprocessing_pipeline import *
import numpy as np
import preprocessing_config as config
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import skimage.measure
import torch
import torch.nn as nn
import torch.optim as optim

### Loading the data

In [17]:
# temp_pipeline = PreProcessingPipelineTemporal()

In [18]:
# train_fn, dataloader_train = temp_pipeline.get_processed_trainloader(64, 'indexbands', vi_type='msi')

Train

In [19]:
temporal_images = load_field_images_temporal(config.base_directory_temporal_train1)
border_removed_images_train = blacken_field_borders_temporal(temporal_images)
field_numbers_train, indices_images_train = allbands_temporal_cubes(border_removed_images_train)

len(indices_images_train), indices_images_train[0][0].shape

(2425, (64, 64, 10))

Evaluation

In [20]:
temporal_images_eval = load_field_images_temporal(config.base_directory_temporal_test1)
border_removed_images = blacken_field_borders_temporal(temporal_images_eval)
field_numbers_eval, indices_images_eval = allbands_temporal_cubes(border_removed_images)

(len(indices_images_eval), indices_images_eval[0][0].shape)

(48, (64, 64, 10))

In [21]:
image_tensor_train = np.stack(indices_images_train)  # Shape: (N x 7 x 64 x 64 x 6)
image_tensor_eval = np.stack(indices_images_eval)   # Shape: (N x 7 x 64 x 64 x 6)

image_tensor_train.shape, image_tensor_eval.shape

((2425, 7, 64, 64, 10), (48, 7, 64, 64, 10))

In [22]:
image_tensor_train = torch.tensor(image_tensor_train, dtype=torch.float32).permute(0, 1, 4, 2, 3)  # (N, T, H, W, C) -> (N, T, C, H, W)
image_tensor_eval = torch.tensor(image_tensor_eval, dtype=torch.float32).permute(0, 1, 4, 2, 3)  # (N, T, H, W, C) -> (N, T, C, H, W)
image_tensor_train.shape, image_tensor_eval.shape

(torch.Size([2425, 7, 10, 64, 64]), torch.Size([48, 7, 10, 64, 64]))

### Mini-patches

In [23]:
train_patches, train_patch_coordinates = non_overlapping_sliding_window(image_tensor_train, field_numbers_train, patch_size=5)
eval_patches, eval_patch_coordinates = non_overlapping_sliding_window(image_tensor_eval, field_numbers_eval, patch_size=5)
len(eval_patches), eval_patches[0].shape

(867, torch.Size([7, 10, 5, 5]))

In [24]:
len(train_patches), len(train_patch_coordinates)

(39042, 39042)

In [25]:
train_patches = np.stack(train_patches)  # Shape: (N x 7 x 6 x 5 x 5)
eval_patches = np.stack(eval_patches)   # Shape: (N x 7 x 6 x 5 x 5)

In [26]:
train_patch_coordinates[0]

('1167134.0', 25, 25)

In [27]:
train_coord_dataloader = field_nos_dataloader(train_patch_coordinates)
eval_coord_dataloader = field_nos_dataloader(eval_patch_coordinates)
train_coord_dataloader[0]

'1167134.0_25_25'

In [28]:
len(train_coord_dataloader)

39042

### Data Loaders

In [29]:
train_ratio = 0.8  

# Split patches and corresponding field numbers
train_patches, test_patches, train_field_numbers, test_field_numbers = train_test_split(
    train_patches, train_coord_dataloader, test_size=1-train_ratio, random_state=42
)

# Create train and test dataloaders
batch_size = 64
dataloader_train = create_data_loader(train_patches, train_field_numbers, batch_size=batch_size, shuffle=True)
dataloader_test = create_data_loader(test_patches, test_field_numbers, batch_size=batch_size, shuffle=False)

for batch_inputs, batch_field_numbers in dataloader_train:
    print("Train Batch Inputs Shape:", batch_inputs.shape)
    print("Train Batch Field Numbers:", batch_field_numbers)
    break  

for batch_inputs, batch_field_numbers in dataloader_test:
    print("Test Batch Inputs Shape:", batch_inputs.shape)
    print("Test Batch Field Numbers:", batch_field_numbers)
    break  

Train Batch Inputs Shape: torch.Size([64, 10, 7, 5, 5])
Train Batch Field Numbers: ('1189784.0_1189788.0_1189793.0_1189795.0_1218453.0_1218735.0_1218738.0_1218750.0_1218753.0_1223918.0_40_35', '1225455.0_15_45', '1224910.0_10_20', '1225307.0_45_25', '1224228.0_30_40', '1227879.0_40_35', '1179940.0_1179941.0_1179944.0_1179949.0_1228193.0_1228196.0_50_10', '1220499.0_1220503.0_1222263.0_1227969.0_1227970.0_1227975.0_40_30', '1218199.0_1218200.0_25_50', '1227724.0_20_40', '1234584.0_30_30', '1219165.0_35_30', '1227426.0_35_30', '1194590.0_35_30', '1169834.0_30_35', '1219168.0_1219170.0_1230929.0_35_30', '1220268.0_30_20', '1226956.0_25_30', '1224910.0_15_30', '1230841.0_20_20', '1224025.0_25_30', '1182585.0_25_40', '1226071.0_15_25', '1194486.0_1195607.0_1226195.0_1226392.0_40_40', '1217783.0_30_25', '1220423.0_30_30', '1223713.0_30_35', '1216570.0_1216574.0_40_50', '1223191.0_1229420.0_40_30', '1228643.0_20_30', '1220522.0_1220523.0_40_45', '1195940.0_30_30', '1221024.0_35_20', '1223563.

In [30]:
batch_size = 64
dataloader_eval = create_data_loader(eval_patches, eval_coord_dataloader, batch_size=batch_size, shuffle=False)

for batch_inputs, batch_field_numbers in dataloader_eval:
    print("Eval Batch Inputs Shape:", batch_inputs.shape) 
    print("Eval Batch Field Numbers:", batch_field_numbers)
    break  

Eval Batch Inputs Shape: torch.Size([64, 10, 7, 5, 5])
Eval Batch Field Numbers: ('1168039.0_20_25', '1168039.0_20_30', '1168039.0_20_35', '1168039.0_25_25', '1168039.0_25_30', '1168039.0_25_35', '1168039.0_30_25', '1168039.0_30_30', '1168039.0_30_35', '1168039.0_35_25', '1168039.0_35_30', '1168039.0_35_35', '1228889.0_25_15', '1228889.0_25_20', '1228889.0_25_25', '1228889.0_25_30', '1228889.0_25_35', '1228889.0_25_40', '1228889.0_30_15', '1228889.0_30_20', '1228889.0_30_25', '1228889.0_30_30', '1228889.0_30_35', '1228889.0_30_40', '1228889.0_35_15', '1228889.0_35_20', '1228889.0_35_25', '1228889.0_35_30', '1168663.0_1176271.0_25_35', '1168663.0_1176271.0_25_40', '1168663.0_1176271.0_30_20', '1168663.0_1176271.0_30_25', '1168663.0_1176271.0_30_30', '1168663.0_1176271.0_30_35', '1168663.0_1176271.0_30_40', '1168663.0_1176271.0_35_20', '1168663.0_1176271.0_35_25', '1168663.0_1176271.0_35_30', '1168663.0_1176271.0_35_35', '1168663.0_1176271.0_35_40', '1168692.0_1220431.0_15_20', '1168692.

### Pre-trained Models: Resnet3D

In [31]:
device = 'cuda'
train_patches.shape

(31233, 7, 10, 5, 5)

In [32]:
resnet3d_extractor = ResNet3DFeatureExtractor()
resnet3d_features_train, train_coord_dl = extract_features(resnet3d_extractor, dataloader_train, device)
print("ResNet3D Extracted Features Shape:", resnet3d_features_train.shape)  

/home/k64835/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/k64835/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet3D Extracted Features Shape: torch.Size([31233, 512])


In [33]:
resnet3d_features_test, test_coord_dl = extract_features(resnet3d_extractor, dataloader_test, device)
print("ResNet3D Extracted Features Shape:", resnet3d_features_test.shape)  

ResNet3D Extracted Features Shape: torch.Size([7809, 512])


In [34]:
resnet3d_features_eval, eval_coord_dl = extract_features(resnet3d_extractor, dataloader_eval, device)
print("ResNet3D Extracted Features Shape:", resnet3d_features_eval.shape)  

ResNet3D Extracted Features Shape: torch.Size([867, 512])


### Evaluation

In [35]:
kmeans = train_kmeans_patches(resnet3d_features_train.cpu(), n_clusters=2, random_state=12)

train_patch_predictions = kmeans.predict(resnet3d_features_train.reshape(resnet3d_features_train.size(0), -1).numpy().astype(np.float32))
test_patch_predictions = kmeans.predict(resnet3d_features_test.reshape(resnet3d_features_test.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(resnet3d_features_eval.reshape(resnet3d_features_eval.size(0), -1).numpy().astype(np.float32))

/home/k64835/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [36]:
# Assign field labels
threshold = 0.5
train_field_labels = assign_field_labels_ae(train_coord_dl, train_patch_predictions, threshold)
test_field_labels = assign_field_labels_ae(test_coord_dl, test_patch_predictions, threshold)
eval_field_labels = assign_field_labels_ae(eval_coord_dl, eval_patch_predictions, threshold)

In [37]:
accuracy, report, x_y_coords = evaluate_test_labels_ae(eval_field_labels, config.labels_path)
print(f"Test Accuracy: {accuracy}")
print(report)

Test Accuracy: 0.5245901639344263
              precision    recall  f1-score   support

           0       0.40      0.23      0.29        26
           1       0.57      0.74      0.64        35

    accuracy                           0.52        61
   macro avg       0.48      0.49      0.47        61
weighted avg       0.49      0.52      0.49        61



In [39]:
convlstm_extractor = ConvLSTMFeatureExtractor()
convlstm_features_train, train_coord_dl = extract_features(convlstm_extractor, dataloader_train, device)
print("ConvLSTM Extracted Features Shape:", convlstm_features_train.shape)


In [ ]:
convlstm_features_test, test_coord_dl = extract_features(convlstm_extractor, dataloader_test, device)
print("ResNet3D Extracted Features Shape:", convlstm_features_test.shape)  

ResNet3D Extracted Features Shape: torch.Size([7809, 512])


In [ ]:
convlstm_features_eval, eval_coord_dl = extract_features(convlstm_extractor, dataloader_eval, device)
print("ResNet3D Extracted Features Shape:", convlstm_features_eval.shape)  

ResNet3D Extracted Features Shape: torch.Size([867, 512])


### Evaluation

In [ ]:
kmeans = train_kmeans_patches(convlstm_features_train.cpu(), n_clusters=2, random_state=12)

train_patch_predictions = kmeans.predict(convlstm_features_train.reshape(convlstm_features_train.size(0), -1).numpy().astype(np.float32))
test_patch_predictions = kmeans.predict(convlstm_features_test.reshape(convlstm_features_test.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(convlstm_features_eval.reshape(convlstm_features_eval.size(0), -1).numpy().astype(np.float32))

/home/k64835/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
# Assign field labels
threshold = 0.5
train_field_labels = assign_field_labels_ae(train_coord_dl, train_patch_predictions, threshold)
test_field_labels = assign_field_labels_ae(test_coord_dl, test_patch_predictions, threshold)
eval_field_labels = assign_field_labels_ae(eval_coord_dl, eval_patch_predictions, threshold)

In [ ]:
accuracy, report, x_y_coords = evaluate_test_labels_ae(eval_field_labels, config.labels_path)
print(f"Test Accuracy: {accuracy}")
print(report)

Test Accuracy: 0.5245901639344263
              precision    recall  f1-score   support

           0       0.40      0.23      0.29        26
           1       0.57      0.74      0.64        35

    accuracy                           0.52        61
   macro avg       0.48      0.49      0.47        61
weighted avg       0.49      0.52      0.49        61

